<a href="https://colab.research.google.com/github/DenisKai7/invoice_generator/blob/main/malam_puncak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install gspread pandas fpdf2 pyqrcode pypng requests pillow
!pip install --upgrade google-auth-oauthlib google-auth-httplib2

# Import necessary libraries
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
from fpdf import FPDF
import os
from google.colab import drive
from datetime import datetime
import requests
from io import BytesIO
from PIL import Image
import urllib.parse
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Authenticate with Google Sheets & Drive
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Initialize Google Drive API
drive_service = build('drive', 'v3', credentials=creds)

# Mount Google Drive to save invoices
drive.mount('/content/drive')

# Open the spreadsheet
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1Eig9wEouZRPDxJvmdvNBJ8g2XYAaXGFPS-bwDtVXcK8/edit?gid=0#gid=0"
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.get_worksheet(0)

# Get all data and check available columns
expected_headers = ['KODE INVOICE', 'Tanggal', 'email', 'KATEGORI', 'NAMA', 'NO WA', 'KODE REFERAL', 'Quantity', 'Barcode', 'template_pesan']
records = worksheet.get_all_records(expected_headers=expected_headers)
df = pd.DataFrame.from_records(records)

# Check actual column names
print("Available columns in spreadsheet:")
print(df.columns.tolist())

# Column mapping
COLUMN_MAPPING = {
    'invoice_id': 'KODE INVOICE',
    'date': 'Tanggal',
    'email': 'email',
    'category': 'KATEGORI',
    'name': 'NAMA',
    'whatsapp': 'NO WA',
    'referral_code': 'KODE REFERAL',
    'quantity': 'Quantity',
    'barcode': 'Barcode',
    'message_template': 'template_pesan'
}

# Path to logos and background image
LOGO_PATH = '/content/drive/MyDrive/logo/diesnatalis 1-01.png'  # Main logo
SECONDARY_LOGO_PATH = '/content/drive/MyDrive/logo/Unipma.png'  # Secondary logo
BACKGROUND_IMAGE_PATH = '/content/drive/MyDrive/logo/gs.png'  # Background image

class InvoicePDF(FPDF):
    def __init__(self, participant_data):
        super().__init__(format='A4', orientation='L')  # A4 Landscape
        self.participant_data = participant_data
        self.add_page()
        self.set_margins(15, 15, 15)  # Left, top, right margins

    def header(self):
        # Header with background color
        self.set_fill_color(230, 230, 230)  # Light gray
        self.rect(0, 0, self.w, 45, 'F')

        # Main logo at top-left
        try:
            if os.path.exists(LOGO_PATH):
                self.image(LOGO_PATH, x=10, y=5, w=50)

                # Display invoice code below the logo
                self.set_xy(10, 25)
                self.set_font('Arial', 'B', 10)
                self.set_text_color(0, 0, 0)  # Black
                invoice_code = self.participant_data.get(COLUMN_MAPPING['invoice_id'], 'INV-XXXXXX')
                self.cell(50, 6, f'Invoice: {invoice_code}', 0, 1, 'L')
        except Exception as e:
            print(f"⚠️ Error loading main logo: {e}")

        try:
            # UNIPMA logo at top-right - adjusted size
            if os.path.exists(SECONDARY_LOGO_PATH):
                self.image(SECONDARY_LOGO_PATH, x=self.w - 30, y=5, w=20)
        except Exception as e:
            print(f"⚠️ Error loading secondary logo: {e}")

        # Concert information at top-right
        self.set_xy(self.w - 80, 10)
        self.set_font('Arial', 'B', 14)
        self.set_text_color(0, 0, 139)  # Dark blue
        self.cell(0, 8, 'GROOVE SPECTA 2025', 0, 1, 'R')

        self.set_xy(self.w - 80, 18)
        self.set_font('Arial', 'I', 10)
        self.set_text_color(0, 0, 0)  # Black
        self.cell(0, 6, '02 Juni 2025', 0, 1, 'R')

        self.set_xy(self.w - 80, 24)
        self.set_font('Arial', '', 9)
        self.set_text_color(100, 100, 100)  # Gray
        self.multi_cell(80, 5, 'Jl. Letkol Suwarno No. 15, Kanigoro, Kec. Kartoharjo, Kota Madiun', 0, 'R')

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.set_text_color(100, 100, 100)
        self.cell(0, 10, 'System By: Jofanza Denis Aldida & Yoga Gondrong', 0, 0, 'C')

    def create_invoice(self):
        # Add semi-transparent background image
        try:
            if os.path.exists(BACKGROUND_IMAGE_PATH):
                self.image(BACKGROUND_IMAGE_PATH, x=0, y=0, w=self.w, h=self.h, type='', link='')
                # Add a white rectangle overlay to make text readable
                self.set_fill_color(255, 255, 255, 200)  # White with transparency
                self.rect(10, 50, self.w - 20, self.h - 70, 'F')
        except Exception as e:
            print(f"⚠️ Error loading background image: {e}")

        # Invoice date at top-right below header
        self.set_y(40)
        self.set_font('Arial', '', 10)
        self.set_text_color(0, 0, 0)
        invoice_date = self.participant_data.get(COLUMN_MAPPING['date'], datetime.now().strftime("%d %b %Y"))
        self.cell(0, 6, f'Issued on: {invoice_date}', 0, 1, 'R')

        # Divider line
        self.line(15, 50, self.w - 15, 50)
        self.ln(15)

        # Participant details section
        self.set_font('Arial', 'B', 14)
        self.set_text_color(0, 0, 139)  # Dark blue
        self.cell(0, 8, 'PARTICIPANT DETAILS', 0, 1, 'L')
        self.ln(5)

        # Participant information table
        self.set_font('Arial', '', 12)
        self.set_text_color(0, 0, 0)

        # Name
        self.set_font('Arial', 'B', 12)
        self.cell(40, 8, 'Name:', 0, 0, 'L')
        self.set_font('Arial', '', 12)
        name = str(self.participant_data[COLUMN_MAPPING["name"]] or '-').replace('’', "'")
        self.cell(0, 8, name, 0, 1, 'L')

        # Category
        self.set_font('Arial', 'B', 12)
        self.cell(40, 8, 'Category:', 0, 0, 'L')
        self.set_font('Arial', '', 12)
        self.cell(0, 8, str(self.participant_data.get(COLUMN_MAPPING['category'], '-')), 0, 1, 'L')

        # Referral Code
        self.set_font('Arial', 'B', 12)
        self.cell(40, 8, 'Referral Code:', 0, 0, 'L')
        self.set_font('Arial', '', 12)
        self.cell(0, 8, str(self.participant_data.get(COLUMN_MAPPING['referral_code'], '-')), 0, 1, 'L')

        self.ln(10)

        # Ticket details table
        # Header
        self.set_fill_color(70, 130, 180)  # Steel blue
        self.set_text_color(255, 255, 255)  # White
        self.set_font('Arial', 'B', 12)
        self.cell(100, 12, 'Description', 1, 0, 'L', 1)
        self.cell(30, 12, 'Quantity', 1, 0, 'C', 1)
        self.cell(40, 12, 'Price', 1, 0, 'R', 1)
        self.cell(40, 12, 'Amount', 1, 1, 'R', 1)

        # Data row
        self.set_text_color(0, 0, 0)  # Black
        self.set_font('Arial', '', 11)
        category = str(self.participant_data.get(COLUMN_MAPPING['category'], '-'))

        # Handle quantity
        quantity_str = str(self.participant_data.get(COLUMN_MAPPING['quantity'], '1')).strip()
        try:
            quantity = int(''.join(filter(str.isdigit, quantity_str))) if quantity_str else 1
        except:
            quantity = 1

        price_per_unit = 100000  # Price per ticket Rp100,000
        price = f"Rp{price_per_unit:,.0f}".replace(",", ".")  # Format with dots
        total_amount = price_per_unit * quantity
        amount = f"Rp{total_amount:,.0f}".replace(",", ".")  # Format with dots

        self.cell(100, 10, f"Groove Specta Ticket - {category}", 'LR', 0, 'L')
        self.cell(30, 10, str(quantity), 'LR', 0, 'C')
        self.cell(40, 10, price, 'LR', 0, 'R')
        self.cell(40, 10, amount, 'LR', 1, 'R')

        # Footer row
        self.cell(100, 0.5, '', 'T', 0, 'L')
        self.cell(30, 0.5, '', 'T', 0, 'C')
        self.cell(40, 0.5, '', 'T', 0, 'R')
        self.cell(40, 0.5, '', 'T', 1, 'R')

        # Total section
        self.ln(5)
        self.set_font('Arial', 'B', 12)
        self.cell(130, 10, 'Subtotal:', 0, 0, 'R')
        self.cell(40, 10, amount, 0, 1, 'R')

        self.cell(130, 10, 'Total:', 0, 0, 'R')
        self.cell(40, 10, amount, 0, 1, 'R')

        paid_date = self.participant_data.get(COLUMN_MAPPING['date'], datetime.now().strftime("%d %b %Y"))
        self.cell(130, 10, f'Paid on {paid_date}', 0, 0, 'R')
        self.cell(40, 10, amount, 0, 1, 'R')

        # Barcode section
        if COLUMN_MAPPING["barcode"] in self.participant_data and self.participant_data[COLUMN_MAPPING["barcode"]]:
            try:
                barcode_url = self.participant_data[COLUMN_MAPPING["barcode"]]
                if barcode_url.startswith('http'):
                    response = requests.get(barcode_url)
                    img = Image.open(BytesIO(response.content))
                    temp_img_path = f"/tmp/barcode_{self.participant_data[COLUMN_MAPPING['invoice_id']]}.png"
                    img.save(temp_img_path)

                    # Position barcode at bottom right
                    self.image(temp_img_path, x=self.w - 70, y=self.h - 50, w=50)
                    os.remove(temp_img_path)

                    # Add text below barcode
                    self.set_xy(self.w - 70, self.h - 20)
                    self.set_font('Arial', '', 8)
                    self.cell(50, 5, 'Scan barcode for verification', 0, 1, 'C')
            except Exception as e:
                print(f"⚠️ Error adding barcode: {e}")

        # Notes section
        self.set_y(self.h - 40)
        self.set_font('Arial', 'I', 9)
        self.set_text_color(70, 70, 70)  # Dark gray
        notes = [
            "Important Notes:",
            "1. This e-invoice serves as your payment proof for ticket redemption.",
            "2. Please present this invoice along with valid ID at the venue entrance.",
            "3. Tickets are non-refundable and non-transferable.",
            "4. For any inquiries, contact us at groovespecta@unipma.ac.id"
        ]

        for note in notes:
            self.cell(0, 5, note, 0, 1, 'L')

def generate_invoice_pdf(participant_data):
    # Create PDF
    pdf = InvoicePDF(participant_data)
    pdf.create_invoice()

    # Create local directory for invoices
    local_invoice_dir = "/content/invoices_groove_specta"
    os.makedirs(local_invoice_dir, exist_ok=True)

    # Save PDF locally
    pdf_filename = f"Groove_Specta_{participant_data[COLUMN_MAPPING['invoice_id']]}.pdf"
    local_pdf_path = f"{local_invoice_dir}/{pdf_filename}"
    pdf.output(local_pdf_path)

    return local_pdf_path, pdf_filename

def upload_to_drive(file_path, file_name, folder_id=None):
    """Upload file to Google Drive and set permissions"""
    file_metadata = {
        'name': file_name,
        'parents': [folder_id] if folder_id else None
    }

    media = MediaFileUpload(file_path, mimetype='application/pdf')

    file = drive_service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id,webViewLink'
    ).execute()

    # Set permissions to make it accessible to anyone with the link
    permission = {
        'type': 'anyone',
        'role': 'reader'
    }

    drive_service.permissions().create(
        fileId=file['id'],
        body=permission
    ).execute()

    return file['webViewLink']

def generate_message_template(participant_data, drive_link):
    # Format WhatsApp number with +62
    whatsapp_num = str(participant_data.get(COLUMN_MAPPING['whatsapp'], ''))
    if whatsapp_num:
        clean_number = ''.join(filter(str.isdigit, whatsapp_num))
        if clean_number.startswith('0'):
            clean_number = '+62' + clean_number[1:]
        elif not clean_number.startswith('62'):
            clean_number = '+62' + clean_number
        else:
            clean_number = '+' + clean_number
    else:
        clean_number = ''

    return f"""
Halo {participant_data[COLUMN_MAPPING['name']]},

Terima kasih telah membeli tiket *Groove Specta 2025*!
Berikut invoice resmi Anda:

🎟️ *Detail Tiket:*
- *Kode Invoice:* {participant_data[COLUMN_MAPPING['invoice_id']]}
- *Kategori:* {participant_data[COLUMN_MAPPING['category']]}
- *Jumlah Tiket:* {participant_data.get(COLUMN_MAPPING['quantity'], '1')}
- *Tanggal Acara:* 02 Juni 2025

📥 *Download Invoice:*
{drive_link}

*Informasi Penting:*
- Harap simpan invoice ini sebagai bukti pembayaran
- Tunjukkan invoice beserta identitas yang valid saat masuk venue
- Invoice ini sah tanpa tanda tangan

Untuk pertanyaan lebih lanjut, hubungi:
📧 groovespecta@unipma.ac.id
📞 +62 812-3456-7890

Salam hangat,
*Panitia Groove Specta 2025*
"""

def process_all_participants():
    updates = []
    errors = []

    # Destination folder ID in Google Drive (replace with your folder ID)
    DRIVE_FOLDER_ID = '1TgkYlvaWh_Wy5heW3zaP6b6WDMbGsxKG'  # Replace with your target folder ID

    for index, row in df.iterrows():
        try:
            invoice_id = row[COLUMN_MAPPING['invoice_id']]
            print(f"\nProcessing participant: {row.get(COLUMN_MAPPING['name'], '')} (Invoice: {invoice_id})")

            # 1. Generate PDF
            local_pdf_path, pdf_filename = generate_invoice_pdf(row)

            # 2. Upload to Google Drive and get link
            drive_link = upload_to_drive(local_pdf_path, pdf_filename, DRIVE_FOLDER_ID)
            print(f"✅ Invoice uploaded to: {drive_link}")

            # 3. Create message template
            message = generate_message_template(row, drive_link)
            updates.append((index, 'template_pesan', message))

            # 4. Clean up local file
            os.remove(local_pdf_path)

        except Exception as e:
            error_msg = f"❌ Error processing {row.get(COLUMN_MAPPING['name'], '')}: {str(e)}"
            print(error_msg)
            errors.append((index, error_msg))

    # Update spreadsheet
    for update in updates:
        row_idx, col_name, value = update
        try:
            # Find matching column
            col_names = [col.lower().replace(' ', '_') for col in df.columns]
            target_col = col_name.lower()

            if target_col in col_names:
                col_idx = col_names.index(target_col) + 1  # +1 because spreadsheet indices start at 1
                worksheet.update_cell(row_idx + 2, col_idx, value)
                print(f"✔️ Updated {col_name} for row {row_idx + 2}")
            else:
                print(f"⚠️ Column {col_name} not found in spreadsheet")
        except Exception as e:
            print(f"❌ Failed to update spreadsheet for row {row_idx}: {str(e)}")

    # Display error summary
    if errors:
        print("\n⛔ Error Summary:")
        for error in errors:
            print(error[1])

# Run main process
process_all_participants()
print("\n✅ Process completed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.7/245.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 1.8 MB/s eta 0:00:00
  Created wheel for pyqrcode: filename=PyQRCode-1.2.1-py3-none-any.whl size=36222 sha256=8ec94751c2e2b77ba1d83bf28e20d5469a18699c3910e1617d1da8933f5463ed
  Stored in directory: /root/.cache/pip/wheels/f5/db/22/ade11e0c0854d1bb2fcc673dc90f6495ee2c0d2b6eb41492e3
Successfully built pyqrcode
Mounted at /content/drive
Kolom yang tersedia di spreadsheet:
['KODE INVOICE', 'Tanggal', '', 'email', 'KATEGORI', 'NAMA', 'NO WA', 'KODE REFERAL', 'Quantity', 'Barcode', 'template_pesan', 'Action']

Memproses peserta: MUHAMMAD RIFA'I (2) (Invoice: A0069KEO)


<ipython-input-1-1f66560dd8d4>:85: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 16)
<ipython-input-1-1f66560dd8d4>:87: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 8, 'INVOICE', 0, 1, 'R')
<ipython-input-1-1f66560dd8d4>:90: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', '', 10)
<ipython-input-1-1f66560dd8d4>:91: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 6, f'#{self.participant_data[COLUMN_MAPPING["invoice_id"]]}', 0, 1, 'R')
<ipython-input-1-1f66560dd8d4>:102: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed


✅ Invoice diupload ke: https://drive.google.com/file/d/1Qj8y4xnlpqyqZABTiT3k2LPpzzyc5ITg/view?usp=drivesdk

Memproses peserta: Putri (Invoice: A0070WYB)
✅ Invoice diupload ke: https://drive.google.com/file/d/1AMPABV2RibBfAARJiUow6jfqpS9mkBPM/view?usp=drivesdk

Memproses peserta: RIZKI BUDI SETIAWAN(1) (Invoice: A0071WYK)
✅ Invoice diupload ke: https://drive.google.com/file/d/1yVzKz3tgFyAODD9KF8vJDfLrrMgwPTSV/view?usp=drivesdk

Memproses peserta: RIFDA KARUNIA MUKTI (Invoice: A0072ELJ)
✅ Invoice diupload ke: https://drive.google.com/file/d/1N69m2xKtGuB6cTDIHGEzvPSlsasGdbT1/view?usp=drivesdk

Memproses peserta: Luqman Ma’ruf Romadhon / Aniputri Nur Fatimah (2) (Invoice: A0073IAI)
✅ Invoice diupload ke: https://drive.google.com/file/d/1BKwLsHGHkCFXz7kZSvdxWL5MB6Y4yWdc/view?usp=drivesdk
✔️ Updated template_pesan untuk baris 2
✔️ Updated template_pesan untuk baris 3
✔️ Updated template_pesan untuk baris 4
✔️ Updated template_pesan untuk baris 5
✔️ Updated template_pesan untuk baris 6

✅ Pro